In [25]:
%matplotlib inline
import pandas as pd
import re 
import csv
import requests
import sys
from io import StringIO

In [2]:
# data from the house
LIMIT = 600
from requests_futures.sessions import FuturesSession

session = FuturesSession(max_workers=5)
futures = [session.get('https://www.govtrack.us/congress/votes/115-2017/h%s/export/csv' 
                       %i) for i in range(LIMIT)]

In [3]:
files = []
for i in futures:
    files.append(i.result().text)

In [56]:
text = [f for f in files if (len(re.findall(r'DOCTYPE html', f)) == 0)]

In [65]:
def create_votes_df(text): # list of text
    expression = re.compile('(.*?)(\s)(#)([0-9]+)(\s)(.*?)(\s-\s)(.*)')
    df = pd.DataFrame()
    for t in text:
        des = t.split('\n')[0]
        sdes = re.match(expression, des)
        temp = pd.read_csv(StringIO(t), skiprows=1)
        temp['chamber'] = sdes.group(1)
        temp['date']  = sdes.group(6)
        temp['number'] = sdes.group(4)
        temp['title'] = sdes.group(8)
        df = df.append(temp)
    return df

In [66]:
house_votes = create_votes_df(text)

In [72]:
# data from the senate
LIMIT = 600

session = FuturesSession(max_workers=5)
futures = [session.get('https://www.govtrack.us/congress/votes/115-2017/s%s/export/csv' 
                       %i) for i in range(LIMIT)]

In [73]:
files = []
for i in futures:
    files.append(i.result().text)

In [74]:
text = [f for f in files if (len(re.findall(r'DOCTYPE html', f)) == 0)]

In [76]:
senate_votes = create_votes_df(text)

In [79]:
senate_votes.shape

In [81]:
votes = house_votes.append(senate_votes)

# Get questions and votes

In [86]:
# senate

url  = 'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml'
response = requests.get(url)
question = re.findall(r'<title>([^>].*[^<])</title>', response.content.decode('utf-8'))
vote_number = re.findall(r'<vote_number>([^>].*[^<])</vote_number>', response.content.decode('utf-8'))
vote_date = re.findall(r'<vote_date>([^>].*[^<])</vote_date>', response.content.decode('utf-8'))

senate = {'question' : question, 'number' : vote_number, 'date' : vote_date}
df_senate = pd.DataFrame(senate)

# house

url  = 'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml'
response = requests.get(url)
question = re.findall(r'<title>([^>].*[^<])</title>', response.content.decode('utf-8'))
vote_number = re.findall(r'<vote_number>([^>].*[^<])</vote_number>', response.content.decode('utf-8'))
vote_date = re.findall(r'<vote_date>([^>].*[^<])</vote_date>', response.content.decode('utf-8'))

senate = {'question' : question, 'number' : vote_number, 'date' : vote_date}
df_senate = pd.DataFrame(senate)



In [117]:
from selenium import webdriver 
import time

driver = webdriver.PhantomJS(executable_path=phantom_path)
driver.get("https://www.govtrack.us/congress/votes") 
time.sleep(3)

NameError: name 'phantom_path' is not defined

In [116]:
driver.close()

NameError: name 'driver' is not defined

In [16]:
print(driver.find_element_by_xpath('//@href').text)

InvalidSelectorException: Message: {"errorMessage":"The result of the xpath expression \"//@href\" is: [object Attr]. It should be an element.","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Connection":"close","Content-Length":"91","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:62304","User-Agent":"Python http auth"},"httpVersion":"1.1","method":"POST","post":"{\"using\": \"xpath\", \"value\": \"//@href\", \"sessionId\": \"aa7472b0-9b1b-11e7-bb4c-2b0fc7a27664\"}","url":"/element","urlParsed":{"anchor":"","query":"","file":"element","directory":"/","path":"/element","relative":"/element","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/element","queryKey":{},"chunks":["element"]},"urlOriginal":"/session/aa7472b0-9b1b-11e7-bb4c-2b0fc7a27664/element"}}
Screenshot: available via screen


In [13]:
print(driver.find_element_by_id("content").text) 
driver.close()

NoSuchElementException: Message: {"errorMessage":"Unable to find element with id 'content'","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Connection":"close","Content-Length":"88","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:62243","User-Agent":"Python http auth"},"httpVersion":"1.1","method":"POST","post":"{\"using\": \"id\", \"value\": \"content\", \"sessionId\": \"22f7fff0-9b1b-11e7-b6ff-532d6bfcaca6\"}","url":"/element","urlParsed":{"anchor":"","query":"","file":"element","directory":"/","path":"/element","relative":"/element","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/element","queryKey":{},"chunks":["element"]},"urlOriginal":"/session/22f7fff0-9b1b-11e7-b6ff-532d6bfcaca6/element"}}
Screenshot: available via screen


In [11]:
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.PhantomJS(executable_path=phantom_path) 
driver.get("") 

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "loadedButton"))) 
finally:
    print(driver.find_element_by_id("content").text)
driver.close()

Here is some important text you want to retrieve!
A button to click!


In [40]:
url = 'https://www.govtrack.us/congress/votes/115-2017/h361/export/csv'

data = pd.read_csv('https://www.govtrack.us/congress/votes/115-2017/h361/export/csv', skiprows=1)
# cr = csv.reader(response)

# for row in cr: 
#     print(row)
# cr = csv.reader(open('https://www.govtrack.us/congress/votes/115-2017/h361/export/csv',"rb"))



# for i in range(100):
#    url = 'https://www.govtrack.us/congress/votes/115-2017/h%s/export/csv' %(i+1)
#    data = r
#    temp = pd.read_csv(url, skiprows = 1)
# data

In [92]:
d = {'1' : 20, '2' :30} 
list(d.items())

[('1', 20), ('2', 30)]